# Explore distributon of inter-event times

The goal of this notebook is to plot the time between antenna connections of individal users aggregated over all users. This can be used inspect the given data e.g. w.r.t. noisy transitions between antennas or w.r.t. very large time deltas that we might want to exclude from the analysis.

Load libaries

In [ ]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime
import matplotlib.pyplot as plt

Specify path to unzipped raw data

In [ ]:
data_files = "../data/0_input_data/calldata/unzipped/*.txt"  # The file pattern to read data from

Initialize spark session and adjust code to available memory space

In [ ]:
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

spark = (
    SparkSession.builder.master("local[*]")
    .config("spark.driver.memory", "16g")
    .appName("mobility_analysis")
    .getOrCreate()
)

schema = (
    StructType()
    .add("time", StringType(), True)
    .add("user", StringType(), True)
    .add("zip1", IntegerType(), True)
    .add("zip2", IntegerType(), True)
    .add("lat", DoubleType(), True)
    .add("lon", DoubleType(), True)
)

Read unzipped calldata

In [ ]:
data = spark.read.csv("../data/0_input_data/calldata/unzipped", sep="|", schema=schema)

Filter out relevant data columns

In [ ]:
preprocessed = data.rdd.map(lambda row: (row["user"], row["time"]))

Groupby data by unique user

In [ ]:
grouped = preprocessed.groupByKey().map(lambda row: row[1].data)

In this step we calculate the inter event time between two antenna connections for each user:

In [ ]:
def extract_time_between_calls(events):
    # Is this the correct sorting criterion?
    sorted_events = sorted(events)
    ret = []
    for a, b in zip(sorted_events[:-1], sorted_events[1:]):
        ret.append(
            (
                datetime.strptime(b, "%Y%m%d%H%M%S")
                - datetime.strptime(a, "%Y%m%d%H%M%S")
            ).seconds
        )
    return ret

Next, we aggregate over the entire data and send the data to the frontend:

In [ ]:
transitions = grouped.flatMap(extract_time_between_calls).collect()

Finally, we plot it:

In [ ]:
fig = plt.hist(transitions)